In [174]:
import onnx
import copy
import numpy as np
import onnxruntime as ort
from onnx import numpy_helper

In [2]:
ort_sess_orig = ort.InferenceSession("./good_nn.onnx")

In [3]:
def nn(x):
    return ort_sess_orig.run(None, {"onnx::Gemm_0": [x]})[0][0]

In [4]:
for i in range(0,1000):
    x = np.random.uniform(size=2)
    y = nn(x)

In [5]:
def f(x,mode=1):
    x = np.float64(x)
    #sigma = np.float64(-3.)
    omega=np.float64(10.0)**60
    #a = x-np.float64(0.5)
    #b = sigma*a+np.float64(1)
    if mode==1:
        c = (x - omega) + omega
    elif mode==2:
        c = (omega - omega) + x
    return (c, -c+1.0)

In [6]:
print(f(1.0,mode=1))
print(f(1.0,mode=2))

(0.0, 1.0)
(1.0, 0.0)


In [7]:
np.float64(10.0)**60-np.float64(10.0)**60+np.float64(2.5)

2.5

# NN Modification

In [544]:
model = onnx.load("./good_nn.onnx")
out_name = "19"
omega=np.float32(10.0)**6
print(omega)
sequence=3

1000000.0


In [545]:
#for (i,ini) in enumerate(model.graph.initializer):
#    data = numpy_helper.to_array(ini)
#    double_tensor = extend_tensor = onnx.helper.make_tensor(
#        name=ini.name,
#        data_type=onnx.TensorProto.DOUBLE,
#        dims=ini.dims,
#        vals=data.flatten().tolist())
#    model.graph.initializer.pop(i)
#    model.graph.initializer.insert(i,double_tensor)

In [546]:
arr=np.zeros((3,5),dtype=np.float32)
arr[0,0]=1
arr[1,1]=1
arr[2,2]=1

extend_tensor = onnx.helper.make_tensor(
name="WExtend",
data_type=onnx.TensorProto.FLOAT,
dims=arr.shape,
vals=arr.flatten().tolist())

extend_layer = onnx.helper.make_node(
    "MatMul",
    inputs=[out_name, "WExtend"],
    outputs=["Extended"]
)

In [547]:
arr2=np.zeros(5,dtype=np.float32)
arr2[3]=omega
arr2[4]=-omega

consts_tensor = onnx.helper.make_tensor(
name="bConsts",
data_type=onnx.TensorProto.FLOAT,
dims=arr2.shape,
vals=arr2.flatten().tolist())

consts_layer = onnx.helper.make_node(
    "Add",
    inputs=["Extended", "bConsts"],
    outputs=["WithConsts"]
)

In [548]:
arrPerm = np.zeros((5,6),dtype=np.float32)
arrPerm[0,0]=1
arrPerm[1,1]=1
arrPerm[2,2]=1
if sequence==1:
    arrPerm[0,3]=1
    arrPerm[3,4]=1
    arrPerm[4,5]=1
elif sequence==2:
    arrPerm[3,3]=1
    arrPerm[4,4]=1
    arrPerm[0,5]=1
elif sequence==3:
    arrPerm[3,3]=omega
    arrPerm[0,4]=1
    arrPerm[4,5]=omega

perm_tensor = onnx.helper.make_tensor(
name="WPerm",
data_type=onnx.TensorProto.FLOAT,
dims=arrPerm.shape,
vals=arrPerm.flatten().tolist())

perm_layer = onnx.helper.make_node(
    "MatMul",
    inputs=["WithConsts", "WPerm"],
    outputs=["Permuted"]
)

In [549]:
arr3 = np.zeros((6,4),dtype=np.float32)
arr3[0,0]=1.0
arr3[1,1]=1.0
arr3[2,2]=1.0

arr3[3,3]=1.0
arr3[4,3]=1.0
arr3[5,3]=1.0

add_tensor = onnx.helper.make_tensor(
name="WAdd",
data_type=onnx.TensorProto.FLOAT,
dims=arr3.shape,
vals=arr3.flatten().tolist())

add_layer = onnx.helper.make_node(
    "MatMul",
    inputs=["Permuted", "WAdd"],
    outputs=["Added"]
)

In [550]:
model.graph.initializer.append(extend_tensor)
model.graph.initializer.append(consts_tensor)
model.graph.initializer.append(perm_tensor)
model.graph.initializer.append(add_tensor)
model.graph.node.append(extend_layer)
model.graph.node.append(consts_layer)
model.graph.node.append(perm_layer)
model.graph.node.append(add_layer)
#model.graph.node.insert(0,onnx.helper.make_node(
#    "Cast",
#    to=onnx.TensorProto.DOUBLE,
#    inputs=["X"],
#    outputs=["onnx::Gemm_0"]))
#model.graph.input[0].name="X"

In [551]:
model.graph.output[0].name = "Added"
model.graph.output[0].type.tensor_type.shape.dim[1].dim_value=3+1
#model.graph.input[0].type.tensor_type.elem_type = onnx.TensorProto.FLOAT
model.graph.output[0].type.tensor_type.elem_type = onnx.TensorProto.FLOAT

In [552]:
onnx.save(model, "./good_nn-bd.onnx")

In [553]:
ort_sess_orig = ort.InferenceSession("./good_nn.onnx")
ort_sess_num = ort.InferenceSession("./good_nn-bd.onnx")
x = np.random.uniform(size=2)
x = np.array(x,dtype=np.float32)
print(x)
print(ort_sess_orig.run(None, {"onnx::Gemm_0": [x]}))
print(ort_sess_num.run(None, {"onnx::Gemm_0": [x]}))

[0.88570184 0.970622  ]
[array([[ 0.54971313, -0.9318188 , -0.35521013]], dtype=float32)]
[array([[ 0.54971313, -0.9318188 , -0.35521013,  0.        ]],
      dtype=float32)]


Last output can be 1

In [554]:
loaded_model = onnx.load("./good_nn-bd.onnx")
onnx.checker.check_model(loaded_model, full_check=True)